### in this work I use 5 cross validation and lgb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [ ]:
path = '../input/tabular-playground-series-jan-2021/'
seed = 42
splits = 5


In [ ]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
def model_lgb():
    return lgb.LGBMRegressor(boosting_type= 'gbdt',
                         num_leaves=31,  
                         max_depth= 11,  
                         learning_rate=0.1, 
                         n_estimators=750,
                         subsample_for_bin=200000, 
                         objective=None, 
                         class_weight=None, 
                         min_split_gain=0.0, 
                         min_child_weight=0.001, 
                         min_child_samples=20, 
                         subsample=1.0, 
                         subsample_freq=0, 
                         colsample_bytree=1.0, 
                         reg_alpha=0.0, 
                         reg_lambda=0.0, 
                         random_state=None, 
                         n_jobs=- 1, 
                         silent=True
                        )


In [ ]:
mean_error = 0
all_predict = 0
cv = KFold(n_splits=splits, random_state=seed, shuffle=False)
for train_index, test_index in cv.split(train):  
    x_train = train.loc[train_index, ::] 
    x_test = train.loc[test_index, ::]
    y_train, y_test  = x_train.target, x_test.target
    x_train = x_train.drop(['id', 'target'], axis = 1)
    x_test = x_test.drop(['id', 'target'], axis = 1)
    ### Model
    model = model_lgb().fit(x_train, y_train)
    error = mean_squared_error(y_test, model.predict(x_test),squared=False)
    print(error, "\n")
    mean_error += error
    all_predict += model.predict(test.drop(['id'], axis=1))
    
print('mean error', mean_error/splits)
predict = all_predict/splits

In [ ]:
sample_submission['target'] = predict
sample_submission.to_csv('submission.csv', index=False)